# A Test Case for Refactoring a Basic Portfolio Optimization Workflow

We recreated a basic portfolio optimization workflow from 
functions that will eventually be called from an automated pipeline.
Then compared to a past project using assets from the 
S&P 500 to ensure proper implimentation. 
We then replaced the current statistical estimators,
which assume a normal distribution, 
with robust estimators like those used in past projects through 202410-202411.
Finally we considered a slight variation that uses a bayesian approch 
to incorperate expectations derived from other sources.
Eventaully the functions called here will become a key part of the 
inital Condor Funds web-based minimally viable prototype.

The goal here is to identify:
* create seperate functions to support a basic portfolio optimization workflow
* call them from this project to comapre against a past project to ensure concistancy
* add alternate stratagies from past projects and compare
* add portfolio class and asset specific stat meta props
* add an alternate bayesian optimization approch and compare 

### *Notice*
*We stress that none of our Jupyter Notebook projects are to be considered final and of publication quality unless otherwise stated. We also provide no warranty or guarantee of any kind. These projects are meant for testing code and learning concepts in a transparent, and often evolving manner. If new methodologies, stratagies or fundamental understandings are applied in the future, they will most likely be explored in future projects. For transparency and documentation, old projects will not be removed but instead will be deprecated (and clearly marked as such). Look for updates on GitHub https://github.com/Rtasseff/condor_test/tree/main/project*

## Setup

In [1]:
# data set
datasetName = 'sp500_combined.csv'

# Paths

# Analytics dir path *USER SET*
analyticsDir = '/Users/rtasseff/projects/condor_test/analytics'
# Data dir path *USER SET*
dataDir = '/Users/rtasseff/projects/tmp/data_analytics_v1'
import sys
# adding analytics to the system path
sys.path.insert(0, analyticsDir)

from data_mining import load
from functions import genStats
from functions import genFin
from functions import assetPreassess as apa
from functions import utils
from functions import portOpt as po
from classes import CondorCoreObs as condor


import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd



## Background 

Educational Source:
https://ocw.mit.edu/courses/15-401-finance-theory-i-fall-2008/dd628e151309a7f23962b1a31b9356e5_MIT15_401F08_lec13.pdf

*** need to review all text ***
Modern Portfolio Theory (MPT) is a concept in Finance that describes ways of diversifying and allocating assets in a financial portfolio in order to maximize the portfolio's expected return given the owner's risk tolerance. American economist Harry Markowitz first introduced MPT in a 1952 paper. The theory was intended to eliminate ideosyncratic risk, which is the risk inherent in a particular investment due to its unique characteristics.

*** At some point want to stress that mathmatically we ae using volitility as a statistical measure for 'risk', but the use of risk has some implied meaning that is not covered by this analysis. then further expand on what it might mean in a later example ***

A key component of this framework is diversification. When using MPT, an investor bundles different types of investments together so that when some of the securities fall in value, other securities rise in equal amount. Thus, the overall portfolio stays even but as markets rise overall, the portfolio rises along with the market's inside tide.

MPT argues that any given investment's risk and return characteristics should not be viewed alone but evaluated by how it affects the overall portfolio's risk and return. That is, an investor can construct a portfolio of multiple assets that will result in greater returns without a higher level of risk. As an alternative, starting with a desired level of expected return, the investor can construct a portfolio with the lowest possible risk that is capable of producing that return.

MPT uses precise financial mathematics to carefully construct the portfolio. The steps involved included:
1. portfolio selection - valuing and choosing the securities that might be included in the portfolio;
2. portfolio optimization - calculating the optimal allocation, or percentage, for each of the selected assets (i.e. choosing the right mix of assets to maximize return and/or minimize risk);
3. portfolio rebalancing - after the portfolio is implimented (i.e. assets are purchased) the value of individual assets will change over time and the holdings of various assets will need to be decreased or increased to ensure the original allocation (i.e. to maintain the correct overall percentage of each asset); 
4. portfolio reallocation - periodically it is important to check on the behaviour of individual assets as changes to the asset allocations may needed.

*** Went off on a tangent here, need to focus this ***
In the analysis here, we focused on step (2). Other steps will be convered in future projects and Condor Funds materials. It is worth pointing out that step (4) is of important practicale concern. Since the analysis here is enterly dependant on historical data, future events may change the statistical estiamtes of this data as they ocurr. In addition, there are many other market factors to consider for investment choices. The importance of these factors depends on the preferences of the individual investor. In particular, the time-frame for investment and the size (number of different assets) of the portfolio impact the liklyhood that observed returns and risks mirror historical ones. Longer investment time-frames and bigger portfolios (e.g. a 'buy-and-hold' straetegy over 10+ years on a well diversifed portfolio) allow for noise and abnormal events to smooth out, or correct themselves, over time. In these cases, reallocation could be done by simply repeating step (2). Shorter time frames (e.g. < 5 years) become increasingly reliant on other market insights not considered here. Of course there is no guarentee on any time-frame, as past performance does not always mean future results. For example, a new technology could drive a once strong company into bankruptcy over the course of a decade. In this case there would be no chance of recovery over time. However, there are stratagies accessible to even novice investors for periodically (sometimes without effort) repeating step (1) as well, but these will not be discussed here. 


Valuing the securities that might be included in the portfolio.
Calculating the desired asset allocation, that is, the mix of assets.
Performing calculations to optimize the portfolio to get the maximum amount of return for the minimum amount of risk.
Using financial analysis to monitor the portfolio to see if it meets expectations and then making changes to the individual securities or asset mix when market warrant a change.
An important consideration in MPT is that based on statistical measures such as variance and correlation, a single investment's performance is less important than how it impacts the entire portfolio.

MPT also assumes that investors are risk-averse, meaning that they prefer a less risky portfolio to a riskier one for a given level of return. As a practical matter, risk aversion implies that most people should invest in multiple asset classes (stocks, bonds, commodities, cash equivalents or cryptocurrencies for example).

## Data

In [ ]:
# Read saved date base


# get all data
assets = load.multiAssetHist_CSV(dataDir+'/'+datasetName)
print('Full data set of assets')
assets
assets.info


In [ ]:
# get subset

# select stocks of interest by symbol list *USER SET*
stocklist = ["MSFT", "NEE", "CVX"]

assetsSub = assets[stocklist]

print('----------------')
print('Subset of assets')
print('----------------')

assetsSub.head()

prices, dates, syms = load.df2np(assetsSub)

print('')
print('- Prices:')
print(prices)

print('')
print('- Dates:')
print(dates)

print('')
print('- Symbols:')
print(syms)

In [ ]:
# optimization

# compare against old results
period=1
returns = genFin.returns(prices,period=period,metric='Log')
# use for spot check
print(np.log(34.95233917/35.0114975))
print(returns)
returnDates = dates[period:]

returnExps, returnDisps = genFin.calc_return_prop(returns,'Normal')

print(returnExps)
print(returnDisps**2)
print(returnDates)
print(len(returns))

"""RTasseff reviewed code and found differences this, and an old code example.
In old example code all rows containing nan values were removed, 
which removes data for all symbols for that date if one symbol has nan.
One example of a major impact this has is that Air BnB was added in 2023.
So there are nans for ABNB for all times before 2023.
A direct comparision to the new workflow could not be made.
We decided against rewriting anything here or there for the sole purpose of 
comparision.
The old code was only used as an example of the method,
no actianable analysis was ever done or planned, it was just a learning example.
We see no reason to spend more time on it.
We did some hand spot checking on the returned values here.
One example is shown.  
More were done in ipython, manually, but not shown here.
We determined this plus previous tests was sufficent to continue.
"""

In [ ]:
import plotly.graph_objects as go

# calc need stat props

# set from 202411_apa_SP500
month=21
sampInt=20


returns = genFin.returns(prices,month,'Relative')
returnDates = dates[month:]

# get sampling subset
inds = np.arange(0,len(returns),sampInt)
returns_samp = returns[inds]
returnDates_samp = returnDates[inds]

returnExps, returnDisps = genFin.calc_return_prop(returns_samp,'Normal')
returnCoDispsSq = genFin.returnCoDispSq(returns_samp,'Normal')
print('Expected Returns')
print(returnExps)
print('Dispersion in Returns')
print(returnDisps) # this should be very near the diagonal of the codispssq
print('Variation in Returns')
print(returnDisps**2) # this should be very near the diagonal of the codispssq
print('Co-Variation in Returns')
print(returnCoDispsSq)


In [ ]:
# -- testing new functions for opt --

# get portfolio properties
m = len(returnExps)
w = np.ones(m) * (1/m)
returnExp_port, returnDisp_port = genFin.asset_set_perform(w, returnExps, returnCoDispsSq)

# mannual spot check with above
print('-- Portfolio Props, Monthly --')
print(returnExp_port)
print(returnDisp_port)

# returns are calculated on monthly basis so use this 
returnExp_port, returnDisp_port = genFin.asset_set_perform(w, returnExps, returnCoDispsSq, annualizeBy='M')

# mannual spot check with above
print('-- Portfolio Props, Annualized --')
print(returnExp_port)
print(returnDisp_port)

# test to capture errors
print('-- Portfolio Props, Errors --')

# catch weight error
w = np.array([.9,.1,.00001])
try:
    returnExp_port, returnDisp_port = genFin.asset_set_perform(w, returnExps, returnCoDispsSq)
except Exception as exc:
    print(exc)

# size mis match
w = np.ones(m+1) * (1/(m+1))
try:
    returnExp_port, returnDisp_port = genFin.asset_set_perform(w, returnExps, returnCoDispsSq)
except Exception as exc:
    print(exc)




# Function to calculate negative Sharpe ratio
w = np.ones(m) * (1/m)
sr = genFin.asset_set_neg_sharpe_ratio(w, returnExps, returnCoDispsSq)
print('-- Sharpe Ratio --')
print(sr)
sr = genFin.asset_set_neg_sharpe_ratio(w, returnExps, returnCoDispsSq, annualizeBy='M')
print(sr)
sr = genFin.asset_set_neg_sharpe_ratio(w, returnExps, returnCoDispsSq, riskFreeRate=.04, annualizeBy='M')
print(sr)


# Function to maximize Sharpe ratio
print('-- Optimize Sharpe Ratio --')
optResults = po.max_sharpe_ratio(returnExps, returnCoDispsSq, riskFreeRate=0)
print(optResults)
optResults = po.max_sharpe_ratio(returnExps, returnCoDispsSq, riskFreeRate=.04,annualizeBy='M')
print(optResults)

# store solutions for later
wSRMax = optResults['x']
print('weights max SR')
print(wSRMax)
allocationSRMax = pd.DataFrame(100*np.round(wSRMax,2),index=syms,columns=["allocation"])
returnSRMax, dispSRMax = genFin.asset_set_perform(wSRMax, returnExps, returnCoDispsSq, annualizeBy='M')

"""did manual sanity check on output.
mostly MSFT as it had the best returns
NEE had slightly lower retunrs but the variance was slightly lower
and the correlation was only moderate, thus some NEE was included
CVX was basically not included as it had low returns and high variation
and the variation was not negative.

We should note that we have made a trade off 
technically the comparision of assets should fully include the exact same 
time frame.
That is the returns and the variance for all asset use the exact same set of 
time points.
However, this may not always be the case since we deal with missing 
variables pairwise and not in totality.
That is if one asset has a missing variable 
we remove that observation from calculating its return and covariances
but we do not remove that observation from calulating another assets
expected return.
We did this to minimize loss of data.  
In an extream example of data loss above, we say that the limited 
data set avalible for one asset, e.g. only a few years of ABNB, 
could mean that we exclude the vast majority of data for all assets 
even though most had data. 
In that example, this mass exclusion meant that we could 
not have enough data fro a proper sample interval simply 
because one asset did not have enough.
On the outset this seems clear that we want to avoide the case above.
And in our code we have.
However, there is a notable draw back in an extream case of negative correlation.
If asset A and B were perfectly uncorrelated (corr coef = -1)
and they both returned positve returns, it would likly be in our interest 
to have both assets.
In fact this is a perfect diversification senario 
such that holding portfolio A and B can completly remove fluctations 
that are specific to A or B, which is what MFT is trying to do overall.
Since they both have positive returns it means that there is something 
increasing in value that is not specfici to A or B and that non-systemic thing is 
what we want to invest in.
It could be overal economic health and/or industry specific health,
but the fact that it is large than one company, is why it is less risky 
numerically less risky because we, by purpose, reduce the variation 
and more generally as one 2 independent companies are less likly to 
experiance sudden and extream loss compared to one company alone.
However, this only works if the negative corealation and 
the positive expected return are calculated on the exact same time frame.
That is we get positive returns in both over the same observations that
they are varing oppositly.
If there was an odd situation in which A had been around for a long time
and until recently did well,
but recently it did very poorly with -x returns.
Where as B only started recently and did well with +x returns.
Maybe B is making only money from otherwise A oppertunities.
Then a situation would arrise where A has an expected positive return y
despite its recent losses because of it long positve history.
But there are not the returns that yeldied the negative corelation 
as it can only be calculated for the recent history in which B existed.
If the overall trend of A holds, as well as the recent neg correlation 
then that is fine, but if the recent trend holds completly 
then the returns will be closer to -x. 
In this case the perfect neg cross corelation just means one 
companies losses and gains earase the others and you are left with 
0 return (no loss but no gain).
This is the case Where A is just being replaced by B.
We are allowing this as it is an extreme outlier case.
And, if one specific asset was so severly comprimised (lots of missing data)
the pipeline would have already thrown a flag alerting the user that 
this asset will comprimise the statistical asset.
We either hurt the analysis by minimizing the data avalible 
with loss of statistical power or by hurting the analysis fundementals.
We need to add an option to min data for more conservitive estimates.
"""


# Function to minimize portfolio Dispersion
print('-- Optimize Dispersion --')

optResults = po.min_dispersion(returnExps, returnCoDispsSq)
print(optResults)
optResults = po.min_dispersion(returnExps, returnCoDispsSq, annualizeBy='M')
print(optResults)

# store solutions for later
wDispMin = optResults['x']
returnDispMin, dispDispMin = genFin.asset_set_perform(wDispMin, returnExps, returnCoDispsSq, annualizeBy='M')
print('weights min disp')
print(wDispMin)
allocationDispMin = pd.DataFrame(100*np.round(wDispMin,2),index=syms,columns=["allocation"])

print(po._asset_set_disp(w, returnExps, returnCoDispsSq, annualizeBy='None'))
print(po._asset_set_disp(w, returnExps, returnCoDispsSq, annualizeBy='M'))
""" Spot checked for sanity,
Dispersion is lower than the individuals (see above) 
some benifit is gained by a combined portfolio beccause they are not perfectly 
correlated.
But you cannot do much better because they are not anit correlated.
We also hold the least amount of CVX because it has the highest dispersion.
Finally we not that it does not mater if you annualize or not 
because it is just multipling by the same constant value.
The function value changes but the position in weight space does not.
The slight difference is just a numerial one.
"""
# Function to optimize the portfolio for a target return
print('-- Optimize for a return--')
# one solution could exist near even weights
optResults = po.min_dispersion(returnExps, returnCoDispsSq, returnTarget=0.17439, annualizeBy='M')
print(optResults)
# one solution could exist near all wieght on worst (third) asset
optResults = po.min_dispersion(returnExps, returnCoDispsSq, returnTarget=0.1101, annualizeBy='M')
print(optResults)
# manual sanity checks confirmed 



In [ ]:
import plotly.graph_objects as go

# Function to efficent frontier 
# get return seris to calculate over 



print(returnExps)
print(np.max(returnExps))

# we will be looking at annualized data for most of going forward
# recall that returns were originally done in months
returnExpsAnn, returnDispsAnn = genFin.annualize(returnExps, returnDisps, 'M')

# the lowest return that has a guaranteed solution is likly at the min variance 
# the highest possible (without leveraging) is 100% w on the max single return
returnTargets = np.linspace(returnDispMin, np.max(returnExpsAnn), 101) #returnSRMax

# the opt method does annualize internally, so send pre annualized
# in future work we need to clean this up
wEF= po.calc_efficient_frontier(returnExps, returnCoDispsSq, returnTargets, riskFreeRate=0.04, annualizeBy='M')

# calculate the properties for each wighting 
# optimization is not exact so while the target return was set as a goal
# it may not have been achived, need to recalculate 
n = len(wEF)
returnExps_EF = np.zeros(n) * np.nan
returnDisps_EF = np.zeros(n) * np.nan

for i in range(n):
    returnExps_EF[i], returnDisps_EF[i] = genFin.asset_set_perform(wEF[i], returnExps, returnCoDispsSq, annualizeBy='M')
    
print('-- Efficient Frontier --')
print('targets')
print(returnTargets[::10])
print('weights')
print(wEF[::10])


# we expectd a smooth transition from the 
# precalculated min disp weights to the 
# precalculated max SR weights and we got it
# cehck



print('r')
print(returnExps_EF[::10])
print('d')
print(returnDisps_EF[::10])

# Asset points

asset_text = [
    f"{sym}: Return: {round(ret * 100, 2)}% +/- points {round(vol * 100, 2)} ({round(vol/ret * 100 * np.sqrt(12)/12, 2)}%)"
    for sym, ret, vol in zip(syms, returnExpsAnn, returnDispsAnn)
]


Asset_points = go.Scatter(
    name="Assets",
    mode="markers",
    x=np.round(returnDispsAnn * 100, 2),
    y=[round(ret * 100, 2) for ret in returnExpsAnn],
    marker=dict(
        color="gray",
        size=14,
        line=dict(
            width=3,
            color="black")),
    text=asset_text,
    hoverinfo='text'
)
#plot the efficient frontier

# Max Sharpe Ratio trace
MaxSharpeRatio = go.Scatter(
    name="Maximum Sharpe Ratio",
    mode="markers",
    x=[dispSRMax * 100],
    y=[round(returnSRMax * 100, 2)],
    marker=dict(
        color="red",
        size=14,
        line=dict(
            width=3,
            color="black")),
    text=[f"Return: {round(returnSRMax * 100, 2)}% +/- {round(dispSRMax * 100, 2)} points ({round((dispSRMax/returnSRMax) * 100, 2)}%)<br>{allocationSRMax.to_string()}"],
    hoverinfo='text'
)

# Min Volatility trace
MinVol = go.Scatter(
    name="Minimum Volatility",
    mode="markers",
    x=[dispDispMin * 100],
    y=[round(returnDispMin * 100, 2)],
    marker=dict(
        color="green",
        size=14,
        line=dict(
            width=3,
            color="black")),
    text=[f"Return: {round(returnDispMin * 100, 2)}% +/- {round(dispDispMin * 100,2)} points ({round((dispDispMin/returnDispMin) * 100, 2)}%)<br>{allocationDispMin.to_string()}"],
    hoverinfo='text'
)

# Efficient Frontier trace
efficient_text = [
    f"Return: {round(ret * 100, 2)}% +/- {round(vol * 100,2)} points ({round((vol/ret) * 100, 2)}%)<br>" +
    "<br>".join([f"{symbol}: {round(weight * 100, 2)}%" for symbol, weight in zip(syms, alloc)])
    for ret, vol, alloc in zip(returnExps_EF, returnDisps_EF, wEF)
]


EF_curve = go.Scatter(
    name="Efficient Frontier",
    mode="lines",
    x= np.round(returnDisps_EF * 100, 2),
    y=[round(ret * 100, 2) for ret in returnExps_EF],
    line=dict(color="black", width=3, dash="dashdot"),
    text=efficient_text,
    hoverinfo='text'
)



riskFreeRate = 0.04
# Risk-Free Asset point
rf_point = go.Scatter(
    name="Risk-Free Asset",
    mode="markers",
    x=[0],  # Assuming risk-free rate has zero volatility
    y=[riskFreeRate * 100],
    marker=dict(
        color="blue",
        size=14,
        line=dict(
            width=3,
            color="black")),
    text=f"Risk-Free Rate: {round(riskFreeRate * 100, 2)}%",
    hoverinfo='text'
)
    
# Capital Allocation Line (CAL)
# include risk free point and SR max AND the corrisponding last point on EF line
# need to move the CAL line calcualation to a general function
def f_cal(x, riskFreeRate, returnSRMax, dispSRMax):
    slope = (returnSRMax - riskFreeRate) / dispSRMax 
    y = x * slope + riskFreeRate
    return (y)
    
cal_x = [0, dispSRMax * 100, returnDisps_EF[-1] * 100]
cal_y = [riskFreeRate * 100, returnSRMax * 100, f_cal(returnDisps_EF[-1], riskFreeRate, returnSRMax, dispSRMax) * 100]
CAL = go.Scatter(
    name="Capital Allocation Line",
    mode="lines",
    x=cal_x,
    y=cal_y,
    line=dict(color="orange", width=2, dash="dashdot")
)

data = [Asset_points, MaxSharpeRatio, MinVol, EF_curve, rf_point, CAL]



layout = go.Layout(
    title='Portfolio Optimization with the Efficient Frontier',
    yaxis=dict(title='Annualized Return (%)'),
    xaxis=dict(title='Annualized Volatility (Points of Return)'),
    showlegend=True,
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        bgcolor="#E2E2E2",
        bordercolor='black',
        borderwidth=2),
    width=660,
    height=495
)

fig = go.Figure(data=data, layout=layout)
fig.show()






In [ ]:
import imp

imp.reload(condor)

# double check the classes built for the portfolio
priceLoader = condor.PriceLoader(dataDir+'/'+datasetName)
print(priceLoader.path)
assetsDF = priceLoader.get_assets_df()
print(assetsDF)

syms = ['MSFT', 'NEE', 'CVX']

assetsDF = priceLoader.get_assets_df(syms=syms)
print(assetsDF)


assets = priceLoader.get_assets(syms=syms)
print(assets)

print(assets[0].sym)
print(assets[1].sym)
print(assets[1].prices.values)



In [ ]:
# the prices above on NEE check to be the same we got before objects

imp.reload(condor)

asset = assets[1]
asset.update_returns()

print(asset.returns.values)
print(asset.returns.name)
print(asset.get_returns_lastUpdated())
print(asset.expectedReturn)
print(asset.returnDispersion)



# for comparision 
print('Asset 1 NEE, period 1 (time frame D), metric log, method normal')
asset.update_returns(timeFrame='D',metric='Log',method='Normal')
print(asset.returns.values)
print(asset.expectedReturn)
print(asset.returnDispersion**2)

In [ ]:
# our spot check of NEE continues to be correct with 
# exp and disp sq equaling above
imp.reload(condor)

m = len(syms)
w = np.ones(m) * (1/m)

portfolio1 = condor.Portfolio(assets,w)

# catch weight error
try:
    _ = condor.Portfolio(assets,w*.1)
except Exception as exc:
    print(exc)

# catch type error
try:
    _ = condor.Portfolio(syms,w)
except Exception as exc:
    print(exc)


portfolio2 = condor.Portfolio(syms,w,priceLoader=priceLoader)

# checking some odd behaviour on comparing classes
print(type(asset))
print(condor.Asset)
print(type(asset)==condor.Asset)
print(isinstance(asset,condor.Asset))
print(str(type(asset)))
print(str(condor.Asset))
print(str(type(asset))==str(condor.Asset))
# must be an issue on how or when they were loaded?



In [ ]:


portfolio1.update_properties(timeFrame='D',metric='Log',method='Normal',annualize=False)
portfolio2.update_properties(timeFrame='D',metric='Log',method='Normal',annualize=False)
print(portfolio1.expectedReturnArray)
print(portfolio2.expectedReturnArray)

In [ ]:
print(portfolio1.prices.values)
print(portfolio2.prices.values)


In [ ]:
# spot check on prices and returns works out

# lets recapture the workflow that leads to the EF

imp.reload(genFin)
imp.reload(condor)
# for now a sample interval of 20 is built into the asset and portfolio by default
# this needs to be imporved, we expect making a Sampler object
# to facilitate multiple functions around data sampling 

# Months as 21 consecutive open trading days and assuming 
# all time courses are a sequence of open trading days
# is built in too.  
# we are ok with this for now, but long term this should all 
# track back to the actual times in the time course without 
# the assumptions

# redo this stuff for the data
priceLoader = condor.PriceLoader(dataDir+'/'+datasetName)

# setup the portfolio (as of now dealing with syms is faster)
portfolio = condor.Portfolio(syms,w,priceLoader=priceLoader)

# update portfolio
portfolio.update_properties(timeFrame='M',metric='Relative',method='Normal',annualize=False)

print('-- Asset Props --')
print(portfolio.expectedReturnArray)
print(portfolio.returnCoDispersionSqMatrix)

print('-- Portfolio Props, Monthly --')
print(portfolio.expectedReturn)
if np.abs(portfolio.expectedReturn - .0145) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')
print(portfolio.returnDispersion)

portfolio.calc_properties(annualize=True)
print('-- Portfolio Props, yearly --')
print(portfolio.expectedReturn)
if np.abs(portfolio.expectedReturn - .1744) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')
print(portfolio.returnDispersion)

print('-- Sharpe --')
sr = portfolio.calc_sharpe_ratio()
print(sr)
if np.abs(sr - .9750) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')

sr = portfolio.calc_sharpe_ratio(annualize=False)
print(sr)
if np.abs(sr - .2814) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')

sr = portfolio.calc_sharpe_ratio(riskFreeRate=0.04, annualize=True)
print(sr)
if np.abs(sr - .7513) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')

print('-- Optimize Sharpe --')
expectedReturn, returnDispersion = portfolio.optimize('Sharpe Ratio', riskFreeRate = 0.04, annualize = True)
print(expectedReturn)
print(returnDispersion)

wOpt = portfolio.weights
print(wOpt)
if np.sum(np.abs(wOpt - np.array([0.8503, 0.1497, 0]))) > 0.0003: # spot check
    raise Exception('missed spotcheck on this')




print('-- Optimize Dispersion --')

expectedReturn, returnDispersion = portfolio.optimize('Dispersion', annualize = True)
print(expectedReturn)
print(returnDispersion)

wOpt = portfolio.weights
print(wOpt)
if np.sum(np.abs(wOpt - np.array([0.36001785, 0.45950151, 0.18048064]))) > 0.0003: # spot check
    raise Exception('missed spotcheck on this')

if np.abs(returnDispersion - 0.17230113829500904) > 0.0001: # spot check
    raise Exception('missed spotcheck on this')



